### Classificateur de texte basique : Embedding & Dropout

Avec la base de données IMBD (50 000 avis sur des films pos/neg)


In [1]:
import os 
import re 
import string

import tensorflow as tf

### Récupération des ensembles de données

In [2]:
batch_size = 32
seed = 1

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

# Ensemble de validation :
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

# Ensemble de test :
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2022-07-04 14:59:42.195616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


- Définition du tokenizer

In [4]:
def custom_standardization(input_data):
    """
    Les ensembes de données contiennent des caractères html, la segmentation basique de TextVect 
    ne permet de les enlever d'où cette segmentation spécifique
    """
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    
    return tf.strings.regex_replace(stripped_html,
    '[%s]' % re.escape(string.punctuation),
    '')

In [5]:
vocab_size = 10000
sequence_length = 200

tokenizer_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)
tokenizer_layer.adapt(raw_train_ds.map(lambda text, label: text)) # Création du vocabulaire (les labels ne sont pas utiles)

In [6]:
print("10 ---> ",tokenizer_layer.get_vocabulary()[10])
print("80 ---> ",tokenizer_layer.get_vocabulary()[80])
print("6458 ---> ",tokenizer_layer.get_vocabulary()[6458])
print('Vocabulary size: {}'.format(len(tokenizer_layer.get_vocabulary())))

10 --->  i
80 --->  bad
6458 --->  whining
Vocabulary size: 10000


### Segmentation des données avant d'entraîner le modèle 
Ce qui revient au même que segmenter les données au fur et à mesure pendant l'entraînement,
c'est juste un peu plus rapide.

In [1]:
# Fonction de segmentation de l'ensemble des données
def ds_tokenizer(text, label):
    """
    Prend en paramètre un élement d'un ensemble de données (texte, label) et le segmente
    avec le tokenizer_layer défini ci-dessus
    """
    
    text = tf.expand_dims(text, -1)
    tokenized_text = tokenizer_layer(text)
    return tokenized_text, label

In [8]:
# On segmente les ensembles de données
train_ds = raw_train_ds.map(ds_tokenizer)
val_ds = raw_val_ds.map(ds_tokenizer)
test_ds = raw_test_ds.map(ds_tokenizer)

# Et on les prépare à l'entraînement
### Sans Autotune c'est environ 2 sec de plus d'entraînement sur la 
### première epoch et une seconde de plus sur toutes les autres, même
### sur une base de données aussi petite et un avec modèle aussi simple.

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
embedding_dim = 100 # Dimension avec laquelle on représente nos jetons

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1),]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 150)         1500150   
                                                                 
 dropout_2 (Dropout)         (None, None, 150)         0         
                                                                 
 global_average_pooling1d_1   (None, 150)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_3 (Dropout)         (None, 150)               0         
                                                                 
 Hi (Dense)                  (None, 1)                 151       
                                                                 
Total params: 1,500,301
Trainable params: 1,500,301
Non-trainable params: 0
____________________________________________

In [15]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    metrics=tf.metrics.BinaryAccuracy(threshold=0.0)
)

In [16]:
epochs = 8  # Pour emb_dim = 100, sur-apprend au-delà de 8 époques

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/8
625/625 [==============================] - 7s 11ms/step - loss: 0.6422 - binary_accuracy: 0.7089 - val_loss: 0.5611 - val_binary_accuracy: 0.7858
Epoch 2/8
625/625 [==============================] - 7s 11ms/step - loss: 0.4748 - binary_accuracy: 0.8255 - val_loss: 0.4229 - val_binary_accuracy: 0.8372
Epoch 3/8
625/625 [==============================] - 7s 11ms/step - loss: 0.3691 - binary_accuracy: 0.8616 - val_loss: 0.3627 - val_binary_accuracy: 0.8542
Epoch 4/8
625/625 [==============================] - 7s 11ms/step - loss: 0.3153 - binary_accuracy: 0.8808 - val_loss: 0.3336 - val_binary_accuracy: 0.8626
Epoch 5/8
625/625 [==============================] - 7s 11ms/step - loss: 0.2802 - binary_accuracy: 0.8945 - val_loss: 0.3171 - val_binary_accuracy: 0.8656
Epoch 6/8
625/625 [==============================] - 7s 11ms/step - loss: 0.2545 - binary_accuracy: 0.9056 - val_loss: 0.3077 - val_binary_accuracy: 0.8700
Epoch 7/8
625/625 [==============================] - 8s 12ms/ste

In [17]:
loss, accuracy = model.evaluate(test_ds)

print(f"Erreur : {loss}")
print(f"Précision: {accuracy}")

782/782 [==============================] - 1s 2ms/step - loss: 0.3207 - binary_accuracy: 0.8668
Erreur : 0.32067352533340454
Précision: 0.8668000102043152


### Exportabilité du modèle 

In [18]:
# Exportation du modèle
export_model = tf.keras.Sequential([
  tokenizer_layer,
  model,
  tf.keras.layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

In [19]:
loss, accuracy = export_model.evaluate(raw_test_ds) # Noter qu'avec la couche de Vect on faire passer l'ensemble de données au format string
print(accuracy)

782/782 [==============================] - 3s 4ms/step - loss: 0.3207 - accuracy: 0.8668
0.8668000102043152
